In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['poems.csv']


In [2]:
file_path="../input/poems.csv"
with open(file_path, 'rb') as f:
  contents = f.read()

contents=contents.decode(encoding='utf-8',errors='ignore')


In [3]:
#print(contents)

In [4]:
from io import StringIO

TESTDATA = StringIO(contents)

dataset = pd.read_csv(TESTDATA, sep=",",error_bad_lines=False)

b'Skipping line 192: expected 4 fields, saw 5\nSkipping line 228: expected 4 fields, saw 5\nSkipping line 290: expected 4 fields, saw 5\nSkipping line 294: expected 4 fields, saw 5\nSkipping line 309: expected 4 fields, saw 5\nSkipping line 393: expected 4 fields, saw 5\n'


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
dataset["category"]=le.fit_transform(dataset["category"])

In [6]:
print(dataset.describe())

         category
count  394.000000
mean     1.494924
std      1.117169
min      0.000000
25%      0.250000
50%      1.500000
75%      2.000000
max      3.000000


In [7]:
print(dataset.iloc[1])

poem_name                                              Lonely Am I 
 poem_content     Lonely are the nights|Lonely are the days|Lone...
poet_name                                                 foulk jim
category                                                          2
Name: 1, dtype: object


In [8]:
#creating our own corpus with the help of word tokenizer, porter stemmer,stopwords, regular expressions and nltk
import re   
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
corpus=[]
ps=PorterStemmer()
stopwords=set(stopwords.words("english"))
for i in range(394):
    review=dataset.iloc[i][1]
    review = re.sub('[^a-zA-Z]', ' ', dataset.iloc[i][1] )
    review = re.sub('[?&#@$%!_.|,-:;"]', ' ', dataset.iloc[i][1] )    
    review=review.lower()
    words=word_tokenize(review)
    d=list()
    for w in words:
        if w not in stopwords:
            d.append(ps.stem(w))
    review=" ".join(d)
    corpus.append(review)
    


In [9]:
#demo of how the corpus looks like
print(corpus[5:6])

["unhappi far thing affair wander along coast lean ridg saw even star go lone ocean black mane wild boar plow snout mal paso mountain old monster snuffl sweet root fat grub slick beetl sprout acorn best nation europ fallen finland star go lone ocean old black bristl boar tear sod mal paso mountain world 's bad way man bound wors mend better lie mountain four five centuri star go lone ocean said old father wild pig plow fallow mal paso mountain keep clear dupe talk democraci dog talk revolut drunk talk liar believ believ tusk long live freedom damn ideolog said gamey black mane boar tusk turf mal paso mountain submit holt"]


In [10]:
#creating bag of word using count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2000)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,3].values

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.30,random_state=0,shuffle=True)

In [12]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, Y_train) 

y_pred = gnb.predict(X_test) 

In [13]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)



In [14]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(Y_test, Y_pred)
cm2 = confusion_matrix(Y_test, y_pred)
#confusion matrix for xgboost in first with accuracy of nearly 90%
print(cm1)
#confusion matrix for naive bayes with low accuracy of nearly 60%
print(cm2)

[[23  1  1  4]
 [ 0 32  1  0]
 [ 2  0 23  6]
 [ 3  3  1 19]]
[[19  0  3  7]
 [10 11  5  7]
 [ 5  3  9 14]
 [ 7  2  3 14]]
